#  Reinforcement Learning (RL)

## **Session 3-3:** Custom environment using Multibody Simulation Code Exudyn 

This example is based on the Exudyn example [OpenAIgymNLinkAdvanced](https://github.com/jgerstmayr/EXUDYN/blob/master/main/pythonDev/Examples/openAIgymNLinkAdvanced.py). Is uses the open-source multibody dynamics code Exudyn for simulating in a _custom_ environment which enables the use of many simulation options including friction models or flexible bodies.  

Authors: Johannes Gerstmayr and Peter Manzl; see also [License](https://github.com/jgerstmayr/EXUDYN/blob/master/LICENSE.txt). 

Training can be visualized using tensorboard: 
* open anaconda prompt (or bash with your Python environment)
* go to current directory / solution
* then call tensorboard as follows:


In [3]:
import os
strTensorboard = 'tensorboard --logdir={}/tensorboard_log'.format(os.getcwd())
print(strTensorboard) # run code to get your current directory

tensorboard --logdir=D:\Work_Uni\WorkshopMLEccomas2025\Session-3-ReinforcementLearning/tensorboard_log



then open [http://localhost:6006/](http://localhost:6006/). 

In [5]:
from exudyn.artificialIntelligence import OpenAIGymInterfaceEnv, spaces, logger
import numpy as np
import sys
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import stable_baselines3

from stable_baselines3.common.logger import Logger # logger for reward
from environmentExudyn import InvertedNPendulumEnv, RewardLoggingCallback, make_env_custom
import tensorboard
from datetime import datetime
flagTensorboard = True  # set to False if you want to print output here instead 

if __name__ == '__main__': 
    # this is only executed when file is direct called in Python; required for multiprocessing. 
    import time
    #use some learning algorithm:
    from stable_baselines3 import A2C, SAC, PPO

### Tensorboard and Logging
    
Tensorboard provides us with live visualization of the training process, e.g. mean reward, cumulative reward per episode, and more. Custom information can also be added, depending on the environment. 


### Create Environment

The environment, implemented in the [environmentExudyn](./environmentExudyn.py), is what the agent is interacting with, in our case the single-link inverted pendulum on the cart. The environment is the MBS simulation model and is created as an object, derived from the OpenAIGymInterfaceEnv from Exudyn. The following functions are required: 
* __step__: Defines how a timestep is performed. The agent's action is mapped to the MBS, the solver is called to integrate one or more timestep(s), check if episode ended, calculate reward and write logging information.
    * _MapAction2MBS_: Defines how the action influences the MBS, e.g. applies a force or changes a setpoint for control.
    * _IntegrateStep_: Calls solver of MBS. 
    * _Output2StateAndDone_: Calculates observation from MBS system state and checks if episode ends ("done"). 
    * _getReward_: Calculates the agent's current reward 
* __reset__: After each episode, the system is reset, initializing the MBS in a new (random) system state. 
* __CreateMBS__: The multibody system is defined and created in this Exudyn-specific function. 



### Create Agent

In this example, the __getModel__ function provides us with different RL methods for the agents: 
* __SAC__: Soft Actor Critic
* __A2C__: Advantage Actor Critic 
* __PPO__: Proximal Policy Optimization  

Note that the batch_size in SAC is a major factor for the computation time per step as SAC is off-policy and uses a buffer. 


In [9]:
    modelType = 'A2C' # use e.g. A2C, SAC or PPO with predefined parameters
    tensorboard_log = None #no logging
    rewardCallback = None
    verbose = 0 #turn off to just view in tensorboard
    if flagTensorboard: #only us if tensorboard is available
        tensorboard_log = "tensorboard_log/" #dir
        rewardCallback = RewardLoggingCallback()
        print('output written to tensorboard, start tensorboard to see progress!')
    else:
        verbose = 1 #turn on without tensorboard
    
    # here the model is loaded (either for vectorized or scalar environment using SAC, A2C or PPO).     
    def getModel(myEnv, modelType='SAC'): 
        if modelType=='SAC': 
            model = SAC('MlpPolicy',
                   env=myEnv,
                   learning_rate=1e-3,
                   device='cpu', #usually cpu is faster for this size of networks
                   batch_size=128,
                   buffer_size=int(10e3),
                   tensorboard_log=tensorboard_log,
                   verbose=verbose)
        elif modelType == 'A2C': 
            model = A2C('MlpPolicy', 
                    myEnv, 
                    device='cpu',  
                    tensorboard_log=tensorboard_log,
                    verbose=verbose)
        elif modelType == 'PPO': 
            model = PPO('MlpPolicy', 
                    myEnv, 
                    device='cpu',  
                    tensorboard_log=tensorboard_log,
                    verbose=verbose)
        else: 
            print('Please specify the modelType.')
            raise ValueError
        return model


output written to tensorboard, start tensorboard to see progress!


### Start training
    
Note that with the implemented  ```flagParallel = True```, _vectorized_ environments are used. Multiple environments run in parallel on different threads/CPU-cores for data generation. Training times strongly depend on the used algorithms and settings and due to race conditions results might not be deterministic - even when setting the seed.  
The value of ```flagContinuous``` can be used to change between continuous and discrete action space ("bang-bang control"). 



In [11]:
    modelName = '{}_cartPole_{}'.format(modelType, datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))
    nSteps = 250e3
    flagContinuous = False
    flagParallel = True
    if not(flagParallel): #'scalar' environment, slower because it uses only one CPU:
        env = InvertedNPendulumEnv(nLinks=1, flagContinuous=flagContinuous)
        # env.TestModel(numberOfSteps=2000, seed=42, sleepTime=0.02, useRenderer=True) # visualize environment by running 2000 steps

        model = getModel(env, modelType=modelType) 
        ts = -time.time()
        print('start learning ', modelName)
        model.learn(total_timesteps=int(nSteps), #min 250k steps required to start having success to stabilize double pendulum
                    # progress_bar=True, #requires tqdm and rich package; set True to only see progress and set log_interval very high
                    log_interval=1, #logs per episode; influences local output and tensorboard
                    callback = rewardCallback,
                    )
        
        print('*** learning time total =',ts+time.time(),'***')
    
        #save learned model
        
        model.save("solution/" + modelName)
   
    else: #parallel; faster #set verbose=0 in getModel()!
        import torch #stable-baselines3 is based on pytorch
        n_cores= max(1,int(os.cpu_count()*1.5//2)) # n_cores is  number of real cores (not threads)
        torch.set_num_threads(n_cores) #seems to be ideal to match the size of subprocVecEnv
        
        print('using',n_cores,'cores')

        from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
        vecEnv = SubprocVecEnv([make_env_custom(flagContinuous=flagContinuous) for i in range(n_cores)])
        
        #main learning task
        model = getModel(vecEnv, modelType=modelType)

        ts = -time.time()
        print('start learning of agent with {}'.format(str(model.policy).split('(')[0]))

        model.learn(total_timesteps=int(nSteps),
                    progress_bar=True, #requires tqdm and rich package; set True to only see progress
                    log_interval=1, # logs per episode; influences local output and tensorboard
                    callback = rewardCallback,
                    )
        print('*** learning time total =',ts+time.time(),'***')
    
        #save learned model
        model.save("solution/" + modelName)
        

using 12 cores


Output()

start learning of agent with ActorCriticPolicy


*** learning time total = 95.90342783927917 ***


### Test Agent

The trained agent is tested in a environment with larger threasholds and visualized. 


In [13]:
if __name__ == '__main__': #  load and test
    if modelType == 'A2C': 
        model = A2C.load("solution/" + modelName)
    elif modelType == 'SAC': 
        model = SAC.load("solution/" + modelName)
    elif modelType == 'PPO':
        model = PPO.load("solution/" + modelName)
        
    env = InvertedNPendulumEnv(thresholdFactor=5, nLinks=1, flagContinuous=flagContinuous) #larger threshold for testing
    
    solutionFile='solution/learningCoordinates.txt'
    env.TestModel(numberOfSteps=1000, model=model, solutionFileName=solutionFile, 
                  stopIfDone=False, useRenderer=False, sleepTime=0) #just compute solution file

    #visualize (and make animations) in exudyn:
    from exudyn.interactive import SolutionViewer
    from exudyn.utilities import LoadSolutionFile
    
    env.SC.visualizationSettings.general.autoFitScene = False
    solution = LoadSolutionFile(solutionFile)
    
    SolutionViewer(env.mbs, solution, timeout=0.005, rowIncrement=2) #loads solution file via name stored in mbs

time spent= 0.8140072822570801
columns imported = [2, 2, 2, 0, 0, 0, 0]
total columns to be imported = 6 , array size of file = 7
